In [14]:
## Import Packages
import warnings
warnings.filterwarnings('ignore')
import os
#os.chdir(os.environ['USERPROFILE'] + '\Documents' + r"\0_Teach\data")
import pandas as pd

In [15]:
## Import Data

df = pd.read_csv('allvariables.csv')
#dropcolumns = ["Close_Honda","Close_Ford","Close_GM","Abs_Transaction","Net_Transaction"]
dropcolumns = ["Abs_Transaction","Net_Transaction"]
new_df = df.drop(dropcolumns,axis=1)
new_df.head()

,Date,Close_NIO,Close_Honda,Close_Toyota,Close_Ford,Close_GM,Net_Transaction_Normalised,Abs_Transaction_Normalised,Prediction_reddit,Prediction_tw,Prediction_sa,Prediction_san,TSLA
0,2020-01-02,3.72,28.639999,142.240005,9.420000,37.380001,0.00000,0.000000,3.0,0.0,-1.0,2.0,86.052002
1,2020-01-03,3.83,28.129999,140.750000,9.210000,36.320000,0.00000,0.000000,4.0,0.0,-1.0,6.0,88.601997
2,2020-01-04,3.78,28.103332,140.756668,9.193333,36.160000,0.00000,0.000000,2.0,0.0,0.0,0.0,89.170664
3,2020-01-05,3.73,28.076666,140.763336,9.176667,36.000000,0.82919,0.023763,4.0,0.0,0.0,0.0,89.739332
4,2020-01-06,3.68,28.049999,140.770004,9.160000,35.840000,0.00000,0.000000,2.0,0.0,-3.0,1.0,90.307999


In [16]:
#Regression with Interaction Variables (collinearity issue considered)

#algorithm is inefficient but clearer for instructional purposes


#df = pd.read_csv('allvariables.csv')
df = new_df
y = 'TSLA'

#get column names:
colname = list(df)
#move y to position 0:
colname.insert(0, colname.pop(colname.index(y)))
df = df[colname]

#transform all Xs into cube & cube-root, using np.cbrt()

trf = ['_square', '_sqrt'] #may be adapted for square & square-root

import numpy as np

for i in list(df)[1:]:
    try:
        df[i + trf[0]] = df[i] ** (3. if '_cube' in trf else 2.)
        df[i + trf[1]] = np.cbrt(df[i]) if '_cube' in trf else np.sqrt(df[i])
    except:
        #column cannot be transformed
        #delete non-numeric column (with no questions asked!):
        df.drop(i, axis=1, inplace=True)

#only numeric columns left
df0 = df.copy() #kept for inclusion of interaction variables later
    
#perform feature selection using AIC

modeleq = ' + '.join(list(df)).replace('+', '~', 1)
#print(modeleq)
maxAIC = np.inf
bmodeleq = modeleq
numx = df.shape[1] - 1
x1x2 = False #interaction variables not yet included

from statsmodels.formula.api import ols

while True:
    regout = ols(modeleq, df).fit()
    AIC = regout.aic
    #see if a better model is found:
    if AIC < maxAIC:
        maxAIC = AIC
        bmodeleq = modeleq

    print('\nAIC =', AIC, 'for', numx, 'Xs.')

    if numx == 1:
        print('Variable left:', modeleq[modeleq.find('~') + 2 :])
        if x1x2:
            #one xvar left
            #get out of 'while' loop:
            break
            
        else:
            #add interaction variables for original untransformed variables in best model so far
            
            numx = bmodeleq.count('+') + 1
            print('\nRestarting from best model (with', numx, 'Xs) found so far...')
            
            colname = bmodeleq.replace('~', '+').split(' + ')
            df = df0[colname]
            colname = colname[1:] #remove y
            
            for i in range(numx):
                #look for 1st transformed variable:
                if colname[i][-5:] in trf:
                    i = i - 1
                    #colname[i] is the last untransformed x
                    break
            
            print('\nAdding', int((i + 1) * i / 2), '2-way interactions among', i + 1,
                  'untransformed variables in best model found so far:')
            for j in range(i):
                #untransformed x in colname up to [i]
                for k in range(j + 1, i + 1):
                    a = colname[j] + '_x_' + colname[k]
                    print(a)
                    df[a] = df[colname[j]] * df[colname[k]] ## Creation of interaction varables
                    
            df0 = df.copy()
            df3 = df.copy()
            #delete any x too highly correlated with another x, to avoid collinearity
            
            corv = pd.DataFrame() #start empty dataframe for corr(Xs, y) to come
            for x in list(df)[1:]:
                #during 1st time thru loop, new column, with label, created in empty dataframe
                corv.loc[x, y] = df[x].corr(df[y]) #new entry, with row label, added to dataframe
                
            corv = corv.loc[abs(corv).sort_values([y]).index, :] #corr(Xs, y) ranked

            delta = 0.3 #corr difference lower limit
            dl2 = []
            icorr = True
            while icorr:
                a = abs(corv).diff() <= delta #adjacent rows with similar abs(corr(Xs, y))
                colname = list(df)[1:]
                dl = []
                print('\nX pairs with correlations >', 1 - delta, ':')
                for b in range(1, a.shape[0]):
                    if a.iloc[b, 0]:
                        if abs(df[a.index[b - 1]].corr(df[a.index[b]])) > 1 - delta:
                            #deleting 1 X from correlated pair:
                            dv0 = a.index[b - 1]
                            dv1 = a.index[b]

                            #neither should already be deleted:
                            if not (dv0 in dl) and not (dv1 in dl):
                                #delete x with rather lower corr(x, y):
                                if abs(corv.loc[dv0, y]) - abs(corv.loc[dv1, y]) >= delta:
                                    d = dv1
                                elif len(dv0) < len(dv1): #delete x with longer name:
                                    d = dv1
                                else:
                                    d = dv0
                                    
                                dl.append(d) #for en masse deletion later
                                corv.drop([d], axis=0, inplace=True) #delete from column of corr with y

                                print(dv0,',',dv1)
            
                if len(dl) > 0:
                    df.drop(axis=1, columns=dl, inplace=True) #variables deleted en masse
                    dl2 = dl2 + dl #keep for real deletion later
                    print('\n' + str(len(dl)), 'variables considered for deletion:')
                    print('\n'.join([str(x) for x in dl]))
                else:
                    print('(no more)')
                    icorr = False
                    
            dl2 = [x for x in dl2] #if x.find('_x_') != -1] #only interaction variables kept
            df0.drop(axis=1, columns=dl2, inplace=True) #collinear interaction variables deleted en masse, for real
            #remaining Xs may be collinear
            print('\n' + str(len(dl2)) + ' interaction variables deleted.')
            
            #potential collinearity issues handled
            
            
            modeleq = ' + '.join(list(df0)).replace('+', '~', 1)
            numx = df0.shape[1] - 1
            x1x2 = True #interaction variables already included
            
            #beyond-pairwise collinearity may still be introduced with the interaction variables
            
            df = df0.copy() #ready for continuing deletion
            continue

    #identify X variable to delete by finding the one with smallest abs(t-stat):
    t = regout.tvalues[1:]
    xdrop = list(t[abs(t) == min(abs(t))].index)[0]
    print('Variable to drop:', xdrop)
    
    df.drop(xdrop, axis=1, inplace=True)
    modeleq = ' + '.join(list(df)).replace('+', '~', 1)
    
    numx = numx - 1

print('\nBest model has', bmodeleq.count('+') + 1, 'Xs:')
out = ols(bmodeleq, df3).fit()
#collinearity is still entirely possible at this stage
print(out.summary2())

print("\nDescending order of X's significance:")
print('\n'.join(list(abs(out.tvalues[1:]).sort_values(0, ascending=False).index)))
#if the single best variable isn't high in above ranking, collinearity might be an issue


AIC = 2772.140100929638 for 33 Xs.
Variable to drop: Net_Transaction_Normalised_sqrt

AIC = 2770.14170626572 for 32 Xs.
Variable to drop: Close_Ford_sqrt

AIC = 2768.172885659872 for 31 Xs.
Variable to drop: Prediction_san_sqrt

AIC = 2824.4010211662944 for 30 Xs.
Variable to drop: Prediction_tw_square

AIC = 2822.6105732047245 for 29 Xs.
Variable to drop: Prediction_tw

AIC = 2821.082189023999 for 28 Xs.
Variable to drop: Prediction_tw_sqrt

AIC = 2819.1354598438475 for 27 Xs.
Variable to drop: Prediction_sa

AIC = 2819.135459843872 for 26 Xs.
Variable to drop: Close_GM_square

AIC = 2817.8020385593577 for 25 Xs.
Variable to drop: Prediction_sa_sqrt

AIC = 4077.92420264248 for 24 Xs.
Variable to drop: Prediction_san_square

AIC = 4075.9321940765203 for 23 Xs.
Variable to drop: Abs_Transaction_Normalised_square

AIC = 4074.12693451426 for 22 Xs.
Variable to drop: Abs_Transaction_Normalised

AIC = 4072.3475534063664 for 21 Xs.
Variable to drop: Close_GM_sqrt

AIC = 4070.6675554944304 f

AIC = 3192.8294277823998 for 54 Xs.
Variable to drop: Prediction_reddit_x_Prediction_tw

AIC = 3190.8346215377865 for 53 Xs.
Variable to drop: Close_Honda_x_Net_Transaction_Normalised

AIC = 3188.8423580618287 for 52 Xs.
Variable to drop: Prediction_sa

AIC = 3188.8423580618287 for 51 Xs.
Variable to drop: Prediction_tw_x_Prediction_sa

AIC = 3186.854122801642 for 50 Xs.
Variable to drop: Prediction_sa_square

AIC = 3184.863819827512 for 49 Xs.
Variable to drop: Prediction_sa_x_Prediction_san

AIC = 3182.87387238184 for 48 Xs.
Variable to drop: Abs_Transaction_Normalised

AIC = 3180.8873328112572 for 47 Xs.
Variable to drop: Prediction_san

AIC = 3178.904554586436 for 46 Xs.
Variable to drop: Prediction_sa_sqrt

AIC = 4582.951903191945 for 45 Xs.
Variable to drop: Net_Transaction_Normalised_x_Prediction_san

AIC = 4580.952402888876 for 44 Xs.
Variable to drop: Abs_Transaction_Normalised_sqrt

AIC = 4578.954245305094 for 43 Xs.
Variable to drop: Prediction_san_square

AIC = 4576.9603954

In [19]:
# Setting Significance Level to 5% and take the significant variables
sig = 0.05
s = out.pvalues.sort_values()
taking = [x for x in s[s<sig].index if x != "Intercept"]
taking

['Close_NIO_sqrt',
 'Close_NIO',
 'Close_NIO_square',
 'Close_Toyota_square',
 'Close_Toyota',
 'Close_Toyota_sqrt',
 'Close_Ford',
 'Close_Honda_square',
 'Close_Honda',
 'Close_Honda_sqrt',
 'Close_Ford_square']

In [55]:
## Export to CSV file
new_df[taking].to_csv("features_0.7.csv")